<a href="https://colab.research.google.com/github/magikarp01/SIFNetflix/blob/master/Predicting_Movie_Reviews_from_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import math

In [44]:
# reading from datasheet

dataframe1 = pd.read_excel("Netflix Dataset Latest 2021.xlsx")
# print(dataframe1)
tags = dataframe1["Tags"]
imdb_scores_init = dataframe1["IMDb Score"]
summaries_init = dataframe1["Summary"]
# print(imdb_scores)

summaries_clean = []
imdb_scores_clean = []

# remove the blank elements
for i in range(len(summaries_init)):
  if summaries_init[i] == None or imdb_scores_init[i] == None:
    continue
  elif math.isnan(imdb_scores_init[i]):
    print("nan")
    continue
  else:
    summaries_clean.append(summaries_init[i])
    imdb_scores_clean.append(imdb_scores_init[i])

nan
nan
nan
nan
nan
nan
nan
nan


In [45]:
# embedding the text of summaries

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [46]:
# format input, output into tensors
summaries = np.array(summaries_clean)
imdb_scores = np.array(imdb_scores_clean)/10
# [summary for summary in summaries])
print(summaries.shape)
print(imdb_scores.shape)
print(summaries)
print(imdb_scores)
# print(np.max(imdb_scores))
print(hub_layer([summaries[3]]))


# for i in range(9425):
#   print(summaries[i])
#   print(f"encoding {i}: {hub_layer([summaries[i]])}")
  
encoded_summaries = np.array([hub_layer([summary]) for summary in summaries])
print(encoded_summaries)
print(encoded_summaries.shape)


(9417,)
(9417,)
['A med student with a supernatural gift tries to cash in on his abilities by facing off against ghosts, till a wandering spirit brings romance instead.'
 'When nerdy Johanna moves to London, things get out of hand when she reinvents herself as a bad-mouthed music critic to save her poverty-stricken family.'
 'After her ex-boyfriend cons her out of a large sum of money, a former bank employee tricks a scam artist into helping her swindle him in retaliation.'
 ...
 'In an idyllic port town on Australias west coast in the summer of 1969, carefree 16-year-old Willie enjoys hanging out with his pals and wooing a beautiful singer named Rosie -- until his mom ships him back to a Catholic boarding school in Perth.'
 'In his third show, Daniël Arends argues that good deeds are a form of self interest, and evil deeds are a hobby.'
 'Madagascar goes wild with holiday spirit in this set of Valentines Day and Christmas-themed tales featuring everyones favorite animal characters.']


In [47]:
model = tf.keras.Sequential()
# model.add()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='relu'))
model.summary()

config = model.get_config() # Returns pretty much every information about your model
print(config["layers"][0]["config"]["batch_input_shape"]) # returns a tuple of width, height and channels

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_6 (Dense)             (None, 16)                816       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________
(None,)


In [48]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [49]:
all_data = np.array(list(zip(summaries, imdb_scores)))
# all_data = tf.data.Dataset.from_tensor_slices((summaries, imdb_scores))

print(all_data.shape)
train_data = all_data[:6000, ]
validation_data = all_data[6000:, ]
print(train_data.shape)
print(validation_data.shape)

(9417, 2)
(6000, 2)
(3417, 2)


In [50]:
# import tensorflow_datasets as tfds

# # Split the training set into 60% and 40% to end up with 15,000 examples
# # for training, 10,000 examples for validation and 25,000 examples for testing.
# train_data, validation_data, test_data = tfds.load(
#     name="imdb_reviews", 
#     split=('train[:60%]', 'train[60%:]', 'test'),
#     as_supervised=True)

# print(train_data)

In [51]:
assert not np.any(np.isnan(imdb_scores))
# assert not np.any(np.isnan(summaries))

In [52]:
# history = model.fit(train_data.shuffle(6000).batch(512),
#                     epochs=10,
#                     validation_data=validation_data.batch(512),
#                     verbose=1)

print(summaries)
# summaries = tf.convert_to_tensor(np.asarray(summaries))
# imdb_scores = tf.convert_to_tensor(imdb_scores)
train_x = summaries[:6000]
train_y = imdb_scores[:6000].astype('float32')
validation_x = summaries[6000:7000]
validation_y = imdb_scores[6000:7000].astype('float32')

history = model.fit(x=train_x, y=train_y,
                    epochs=10,
                    validation_data=(validation_x, validation_y),
                    verbose=1)

['A med student with a supernatural gift tries to cash in on his abilities by facing off against ghosts, till a wandering spirit brings romance instead.'
 'When nerdy Johanna moves to London, things get out of hand when she reinvents herself as a bad-mouthed music critic to save her poverty-stricken family.'
 'After her ex-boyfriend cons her out of a large sum of money, a former bank employee tricks a scam artist into helping her swindle him in retaliation.'
 ...
 'In an idyllic port town on Australias west coast in the summer of 1969, carefree 16-year-old Willie enjoys hanging out with his pals and wooing a beautiful singer named Rosie -- until his mom ships him back to a Catholic boarding school in Perth.'
 'In his third show, Daniël Arends argues that good deeds are a form of self interest, and evil deeds are a hobby.'
 'Madagascar goes wild with holiday spirit in this set of Valentines Day and Christmas-themed tales featuring everyones favorite animal characters.']
Epoch 1/10
188/1